Here’s a complete guide to creating a bootable USB drive using OSDCloud and Windows Configuration Designer (WCD) to install the NinjaOne agent during the Windows provisioning process. This guide is an adaptations of an [artice](https://www.ninjaone.com/blog/zero-touch-device-provisioning/) written by Jeff Hunter at NinjaOne and includes all the necessary steps:

### **Step 1: Prepare Your Environment**


The below script block will download any of the necessary scripts to a new folder on your C: drive named 'Provisioning':

In [ ]:

# Predefined array of script URLs and corresponding file names
$ScriptUrls = @(
    "https://raw.githubusercontent.com/Kalichuza/ninjaAdminScripts/main/Windows-ImageDeployment/Prep-USB.ps1",
    "https://github.com/Kalichuza/ninjaAdminScripts/raw/main/Windows-ImageDeployment/New-NinjaWinPeUsb.ps1",
    "https://github.com/Kalichuza/ninjaAdminScripts/raw/main/Windows-ImageDeployment/unattend.xml"
)

$FileNames = @(
    "Prep-USB.ps1",
    "New-NinjaWinPeUsb.ps1",
    "unattend.xml"
)

# Create a folder named "provisioning" in the root C: drive
$provisioningFolder = "C:\Provisioning"
if (-Not (Test-Path -Path $provisioningFolder)) {
    New-Item -Path $provisioningFolder -ItemType Directory -Force
}

# Ensure the number of URLs matches the number of file names
if ($ScriptUrls.Length -ne $FileNames.Length) {
    Write-Error "The number of script URLs must match the number of file names."
    exit 1
}

# Loop through each URL and download the corresponding script
for ($i = 0; $i -lt $ScriptUrls.Length; $i++) {
    $url = $ScriptUrls[$i]
    $fileName = $FileNames[$i]
    $outFile = Join-Path -Path $provisioningFolder -ChildPath $fileName

    # Downloads remotely hosted code and saves it as an outfile of choosing
    try {
        Invoke-WebRequest -Uri $url -OutFile $outFile
        Write-Host "Downloaded $fileName successfully from $url to $provisioningFolder"
    } catch {
        $errorMessage = $_.Exception.Message
        Write-Error "Failed to download `$fileName` from `$url`: $errorMessage"
    }
}


**Skipping The OOBE Using unattend.xml**
- You could use the cookbook located here:
[Unattennded install xml Generator](https://schneegans.de/windows/unattend-generator/)

- You could also use the included file in this repo. Here’s a clear list of all the values in the `unattend.xml` file that you will need to customize:

### Customization Instructions:
1. Replace `examplepassword123` with your desired user password.
2. Replace `ExampleUser` with the desired username.
3. Replace `Primary User Account` with a description for the user account (optional).
4. Replace `John Doe` with the registered owner name.
5. Replace `ExampleCorp` with the registered organization name (optional).

- Place the Final file into the root WinPE directory
This ensures that the `unattend.xml` is tailored to your specific deployment needs.
#### **Install Prerequisites**

1. **Windows Assessment and Deployment Kit (ADK)**:
   - Download and install the Windows ADK from the [official Microsoft site](https://learn.microsoft.com/en-us/windows-hardware/get-started/adk-install).

In [ ]:
#!pwsh
# Define the URLs for the ADK and WinPE Add-on installers
$adkUrl = "https://go.microsoft.com/fwlink/?linkid=2166053"  # This link is for ADK for Windows 11, version 22H2

# Define the download paths
$adkInstallerPath = "$env:TEMP\adksetup.exe"

# Download the ADK installer
Invoke-WebRequest -Uri $adkUrl -OutFile $adkInstallerPath

# Define installation directories (optional)
$adkInstallPath = "C:\Program Files (x86)\Windows Kits\10"

# Install the ADK with selected features
Start-Process -Wait -FilePath $adkInstallerPath -ArgumentList `
    "/quiet", `
    "/features OptionId.ApplicationCompatibilityToolkit `
               OptionId.DistributedScanManagementOptionId.CompatibilityAdministrator `
               OptionId.DeviceExperience `
               OptionId.WindowsPerformanceToolkit `
               OptionId.ApplicationCompatibilityToolkitsOptionId.WindowsPerformanceRecorder `
               OptionId.WindowsPerformanceAnalyzer `
               OptionId.AssessmentToolkit `
               OptionId.WindowsAssessmentToolkit `
               OptionId.DeploymentTools `
               OptionId.WindowsPreinstallationEnvironment `
               OptionId.UserStateMigrationTool `
               OptionId.VolumeActivationManagementTool `
               OptionId.WindowsImageConfigurationOptionId.MSI Wrapper `
               OptionId.MediaExperience `
               OptionId.NetworkExperience `
               OptionId.Networking `
               OptionId.PressPass `
               OptionId.ScanningTools `
               OptionId.SecurityTools `
               OptionId.StorageExperience", `
    "/installpath $adkInstallPath"






2. **WinPE Add-on**:
   - Install the Windows PE add-on, which provides the necessary tools for creating the WinPE environment.

In [ ]:

# Define the URLs for the ADK and WinPE Add-on installers
$winPeUrl = "https://go.microsoft.com/fwlink/?linkid=2166060"  # This link is for the WinPE add-on for Windows ADK for Windows 11, version 22H2

# Define the download paths
$winPeInstallerPath = "$env:TEMP\winpesetup.exe"

# Download the ADK installer
Invoke-WebRequest -Uri $adkUrl -OutFile $adkInstallerPath

# Download the WinPE installer
Invoke-WebRequest -Uri $winPeUrl -OutFile $winPeInstallerPath

# Define installation directories (optional)
$winPeInstallPath = "C:\Program Files (x86)\Windows Kits\10\WinPE"

# Install the WinPE add-on
Start-Process -Wait -FilePath $winPeInstallerPath -ArgumentList "/quiet", "/installpath $winPeInstallPath"




3. **Install Windows Configuration Designer**:
   - This is available [Here](https://www.microsoft.com/store/productId/9NBLGGH4TX22)
   - Detailed instructions on how to create a custom .ppkg file can be found [Here.](https://github.com/Kalichuza/ninjaAdminScripts/tree/main/Windows-ConfigurationDesigner)

   4. **Visual Studio Code with PowerShell Extension**:
   - Download and install [VS Code](https://code.visualstudio.com/) and the PowerShell extension from the Extensions Marketplace.
   - Or run the auto-install script below:


In [ ]:
#!pwsh
# Define the URL for the latest VS Code installer
$vscodeUrl = "https://update.code.visualstudio.com/latest/win32-x64-user/stable"

# Define the download path
$installerPath = "$env:TEMP\VSCodeSetup.exe"

# Download the VS Code installer
Invoke-WebRequest -Uri $vscodeUrl -OutFile $installerPath

# Install VS Code silently
Start-Process -Wait -FilePath $installerPath -ArgumentList "/verysilent" -NoNewWindow

# Clean up the installer after installation
Remove-Item -Path $installerPath -Force

Write-Output "Visual Studio Code installation completed successfully."



#### **Prepare the USB Drive**
- Ensure you have a USB drive with at least 32 GB of space.
- Insert the USB drive into your computer.

#### **Set Up the OSDCloud Workspace**
1. **Run PowerShell Script New-NinjaWinPeUsb.ps1**:
 - Place the File in your downloads folder 
 - This will create the usb where you will place your customization files

In [ ]:
#Ensure the execution policy is bypassed
Set-ExecutionPolicy -ExecutionPolicy Bypass -Scope Process

Ivoke-expression 'C:\Provisioning\New-NinjaWinPeUsb.ps1'

#### **Set Up the OSDCloud/WinPE Usb**
Before running th following code, check the letter drive for the WinPE partition and makes sure to fill in the variable below with the correct letter:


In [ ]:
$WinPE = "E:"
mkdir -Path $WinPE\OSDCloud\Automate\Provisioning\


### **The next step will be to copy the needed files on to the new USB:**


In [ ]:
cd C:\Provisioning

.\Prep-USB.ps1 -WinPEPathProvisioning "<fill in USB WinPE OSDDrive here>" -UnattendXMLPathUSB "<fill in WinPE Root here>" -ProvisioningFolderPath "<Path to your provisioning files>" -UnattendXMLPathProvisioning "<Path to your unattend.xml file>"

### **After completing these steps, you will have a bootable Windows USB**

- Select the USB in the boot menu and walk away. The rest will be taken care of by the script.